In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/students-drop-out-prediction/train.csv
/kaggle/input/students-drop-out-prediction/test.csv


In [2]:
# Loading data

train=pd.read_csv('../input/students-drop-out-prediction/train.csv')
test=pd.read_csv('../input/students-drop-out-prediction/test.csv')
print(train.shape)
print(test.shape)

(3796, 43)
(1628, 42)


In [3]:
train.head()

,id,v_21,v_32,v_15,v_4,v_1,v_8,v_12,v_25,v_20,...,v_2,v_16,v_34,v_35,v_36,v_37,v_38,v_39,v_40,label
0,0,3,6,11,55,1,1,16,5,60,...,-0.3,0.79,1,93,47,1,48,26,0,1
1,1,8,94,12,65,1,4,22,4,173,...,5.4,6.51,6,29,37,25,53,35,7,2
2,2,3,166,10,155,1,1,16,1,309,...,0.6,2.02,1,8,34,4,43,38,2,1
3,3,3,6,13,121,1,1,16,1,6,...,0.6,2.02,7,37,40,27,43,65,0,1
4,4,4,51,18,21,1,17,22,118,168,...,8.4,12.51,16,22,25,25,24,23,1,1


In [4]:
test.head()

,id,v_21,v_32,v_15,v_4,v_1,v_8,v_12,v_25,v_20,...,v_7,v_2,v_16,v_34,v_35,v_36,v_37,v_38,v_39,v_40
0,0,9,90,15,121,1,8,23,114,256,...,10.8,10.4,6.74,4,96,36,34,23,72,5
1,1,7,123,14,126,1,5,19,3,40,...,19.4,0.5,6.79,15,78,12,25,27,67,12
2,2,3,6,10,102,1,1,16,181,168,...,7.6,2.6,0.32,4,81,47,30,54,23,4
3,3,3,6,10,148,1,1,16,110,278,...,11.1,0.6,2.02,4,100,6,23,12,28,1
4,4,3,6,10,148,1,1,16,1,168,...,7.6,2.6,0.32,7,97,25,30,18,52,0


In [5]:
X_train=train.drop('label',axis=1)
y_train=train['label']
X_test=test

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3796 entries, 0 to 3795
Data columns (total 43 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      3796 non-null   int64  
 1   v_21    3796 non-null   int64  
 2   v_32    3796 non-null   int64  
 3   v_15    3796 non-null   int64  
 4   v_4     3796 non-null   int64  
 5   v_1     3796 non-null   int64  
 6   v_8     3796 non-null   int64  
 7   v_12    3796 non-null   int64  
 8   v_25    3796 non-null   int64  
 9   v_20    3796 non-null   int64  
 10  v_18    3796 non-null   int64  
 11  v_3     3796 non-null   int64  
 12  v_11    3796 non-null   int64  
 13  v_14    3796 non-null   int64  
 14  v_26    3796 non-null   int64  
 15  v_27    3796 non-null   int64  
 16  v_30    3796 non-null   int64  
 17  v_9     3796 non-null   int64  
 18  v_31    3796 non-null   int64  
 19  v_28    3796 non-null   int64  
 20  v_13    3796 non-null   int64  
 21  v_33    3796 non-null   int64  
 22  

In [7]:
print(list(train.isna().sum()))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
s=StandardScaler()
scaled_X_train=s.fit_transform(X_train)
scaled_X_test=s.fit_transform(test)

# baseline model 
from sklearn.dummy import DummyClassifier
# strategy{“most_frequent”, “prior”, “stratified”, “uniform”, “constant”}, default=”prior”
dummy_clf = DummyClassifier(random_state=42)
dummy_clf.fit(scaled_X_train,y_train)
f1_score(y_train,dummy_clf.predict(scaled_X_train),average='micro')

0.49077976817702845

In [9]:
from sklearn.linear_model import LogisticRegressionCV
lr=LogisticRegressionCV(cv=5,multi_class='multinomial', random_state=42)
lr.fit(scaled_X_train,y_train)
lr.score(scaled_X_train,y_train)
f1_score(y_train,lr.predict(scaled_X_train),average='micro')

0.7442044257112751

In [10]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(scaled_X_train)

lr_p=LogisticRegressionCV(cv=5, multi_class='multinomial', random_state=42, max_iter=100)
lr_p.fit(X_poly,y_train)
lr_p.score(X_poly,y_train)
f1_score(y_train,lr_p.predict(X_poly),average='micro')

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


0.8350895679662803

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

param_grid = {'n_neighbors':[2,5,9,12],
              'weights':['uniform','distance'],
              'algorithm':['ball_tree', 'kd_tree', 'brute'],
              'metric':['euclidean', 'manhattan']}

knn = KNeighborsClassifier()

grid_knn=GridSearchCV(knn,param_grid = param_grid,cv=5,verbose=True)
grid_knn.fit(scaled_X_train,y_train)
print(grid_knn.best_params_)
f1_score(y_train,grid_knn.predict(scaled_X_train),average='micro')

Fitting 5 folds for each of 48 candidates, totalling 240 fits
{'algorithm': 'ball_tree', 'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'}


0.7407797681770284

In [12]:
from sklearn.svm import SVC

param_grid = {'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
              'C':[10,1,0.1,0.01],
              'gamma':['scale','auto']}

svc = SVC(random_state=42)

grid_svc=GridSearchCV(SVC(),param_grid=param_grid,cv=5,verbose=True)
grid_svc.fit(scaled_X_train,y_train)
print(grid_svc.best_params_)
f1_score(y_train,grid_svc.predict(scaled_X_train),average='micro')

Fitting 5 folds for each of 32 candidates, totalling 160 fits
{'C': 1, 'gamma': 'scale', 'kernel': 'linear'}


0.7481559536354057

In [13]:
from sklearn.tree import DecisionTreeClassifier

param_grid = {'splitter':['best','random'],
              'criterion':['gini','entropy'],
              'max_depth':[3,5,7],
              'min_samples_split':[2,4,6],
              'max_features':['sqrt','log2',None]}

dtc = DecisionTreeClassifier(random_state=42)

grid_dtc=GridSearchCV(dtc,param_grid=param_grid,cv=5,verbose=True)
grid_dtc.fit(scaled_X_train,y_train)
print(grid_dtc.best_params_)
f1_score(y_train,grid_dtc.predict(scaled_X_train),average='micro')

Fitting 5 folds for each of 108 candidates, totalling 540 fits
{'criterion': 'gini', 'max_depth': 5, 'max_features': None, 'min_samples_split': 6, 'splitter': 'best'}


0.7457850368809275

In [14]:
from sklearn.ensemble import RandomForestClassifier

param_grid = {'n_estimators':[50,100,150],
              'criterion':['gini','entropy'],
              'min_samples_split':[2,4,6]}

rfc = RandomForestClassifier(random_state=42)

grid_rfc=GridSearchCV(rfc,param_grid=param_grid,cv=5,verbose=True)
grid_rfc.fit(scaled_X_train,y_train)
print(grid_rfc.best_params_)
f1_score(y_train,grid_rfc.predict(scaled_X_train),average='micro')

Fitting 5 folds for each of 18 candidates, totalling 90 fits
{'criterion': 'gini', 'min_samples_split': 6, 'n_estimators': 150}


0.9799789251844047

In [15]:
from sklearn.ensemble import BaggingClassifier

b1_clf = BaggingClassifier(base_estimator=svc, n_estimators=50, random_state=42).fit(scaled_X_train,y_train)
f1_score(y_train,b1_clf.predict(scaled_X_train),average='micro')

0.8350895679662803

In [16]:
b2_clf = BaggingClassifier(base_estimator=dtc, n_estimators=50, random_state=42).fit(scaled_X_train,y_train)
f1_score(y_train,b2_clf.predict(scaled_X_train),average='micro')

0.9997365648050579

In [17]:
b3_clf = BaggingClassifier(base_estimator=rfc, n_estimators=50, random_state=42).fit(scaled_X_train,y_train)
f1_score(y_train,b3_clf.predict(scaled_X_train),average='micro')

0.9789251844046365

In [18]:
from sklearn.ensemble import AdaBoostClassifier

param_grid = {'n_estimators':[50,100],
              'learning_rate':[0.5,1,2]}

abc = AdaBoostClassifier(random_state = 42)

grid_abc=GridSearchCV(abc,param_grid=param_grid,cv=5,verbose=True)
grid_abc.fit(scaled_X_train,y_train)
print(grid_abc.best_params_)
f1_score(y_train,grid_abc.predict(scaled_X_train),average='micro')

Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'learning_rate': 0.5, 'n_estimators': 100}


0.7481559536354057

In [19]:
# XGBoost

In [20]:
# # MLP
from sklearn.neural_network import MLPClassifier

param_grid = {'solver':['adam','lbfgs'],
              'learning_rate':['constant','adaptive','invscaling'],
              'activation':['identity', 'logistic', 'tanh', 'relu']
             }

MLP_clf = MLPClassifier(random_state=42, warm_start=True)

grid_mlp=GridSearchCV(MLP_clf,param_grid=param_grid,cv=5,verbose=True)
grid_mlp.fit(scaled_X_train,y_train)
print(grid_mlp.best_params_)
f1_score(y_train,grid_mlp.predict(scaled_X_train),average='micro')

Fitting 5 folds for each of 24 candidates, totalling 120 fits


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multi

{'activation': 'logistic', 'learning_rate': 'constant', 'solver': 'adam'}


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


0.797154899894626

In [21]:
preds = grid_rfc.predict(scaled_X_test)
test['label'] = preds
df = {'id': test['id'], 'label': test['label']}
df = pd.DataFrame(data=df)
df.to_csv('submission.csv',index=False)

In [22]:
df

,id,label
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
1623,1623,0
1624,1624,1
1625,1625,0
1626,1626,0
